In [1]:
!pip install numpy
!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install pytorch_lightning==0.8.4
!pip install comet-ml
!pip install torchsummary
!pip install optuna
!pip install joblib

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 307kB 4.6MB/s 
     |████████████████████████████████| 829kB 14.6MB/s 
     |████████████████████████████████| 276kB 28.6MB/s 
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=eb38e0a4c95bc6a976dbd13c599723c6efce24954138d03135b50b72ca077bdb
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44619 sha256=e194427fb5b402f7ab720eacdfab164c63d297242235be9e81d046143c4c41c8
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built future PyYAML
  Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      Successfully uninstalled future-0.16.0
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Su

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Fri Aug 28 18:03:45 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# Mount your personal google drive
from google.colab import drive 
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
primary_directory = '/content/drive/My Drive/KBP_Challenge'
%cd /content/drive/My\ Drive/KBP_Challenge
%ls

/content/drive/My Drive/KBP_Challenge
apex/         data/            pretrained_models/  setup.sh  study.pkl
checkpoints/  lightning_logs/  provided_code/      src/


In [5]:
import torch
import pytorch_lightning as pl
from pytorch_lightning import Callback
from pytorch_lightning.callbacks import LearningRateLogger
from pytorch_lightning.logging import CometLogger
from src.models.pix2pix_model import Pix2PixModel
from pytorch_lightning.callbacks import ModelCheckpoint
from src.options.train_options import TrainOptions
from torchsummary import summary
import optuna
from optuna.integration import PyTorchLightningPruningCallback
import joblib

In [6]:
class MetricsCallback(Callback):
    """PyTorch Lightning metric callback."""

    def __init__(self):
        super().__init__()
        self.metrics = []

    def on_validation_end(self, trainer, pl_module):
        self.metrics.append(trainer.callback_metrics)

In [7]:
args = ['--batchSize', '8',
        '--primary_directory', primary_directory,
        '--which_model_netG', 'unet_128_3d',
        '--which_model_netD', 'n_layers_3d',
        '--n_layers_D', '3',
        '--num_D', '1',
        '--norm_D', 'spectralbatch',
        '--norm_G', 'spectralbatch',
        '--resnet_depth', '18',
        '--which_direction', 'AtoB',
        '--input_nc', '1',
        '--lambda_A', '100',
        '--lr_policy', 'none',
        '--epoch_count', '15',
        '--load_epoch', '-1',
        '--lr_D', '0.0004',
        '--lr_G', '0.0001',
        '--lr_max', '0.01',
        '--lr_step_size', '25',
        '--loss_function', 'smoothed_L1',
        '--init_type', 'xavier',
        #'--no_scaling',
        # '--no_normalization',
        '--no_perceptual_loss',
        '--patience', '5',
        '--n_critic', '1',
        '--n_generator', '3',
        '--cut_blur_mask', '50',
        '--weight_cliping_limit', '0.1',
        '--inst_noise_sigma', '0',
        '--inst_noise_sigma_iters', '1']

opt = TrainOptions().parse(args)

------------ Options -------------
batchSize: 8
beta1: 0.5
beta2: 0.999
checkpoints_dir: ./checkpoints
continue_train: False
cut_blur_mask: 50
dataset_mode: unaligned
display_freq: 200
display_id: 1
display_port: 8097
display_single_pane_ncols: 0
display_winsize: 128
epoch_count: 15
fineSize: 128
gpu_ids: -1
init_type: xavier
input_nc: 1
inst_noise_sigma: 0.0
inst_noise_sigma_iters: 1
isTrain: True
lambda_A: 100.0
lambda_B: 10.0
lambda_identity: 0.5
lambda_perceptual: 10.0
loadSize: 128
load_epoch: -1
loss_function: smoothed_L1
lr_D: 0.0004
lr_G: 0.0001
lr_decay_iters: 50
lr_max: 0.01
lr_policy: none
lr_step_size: 25
max_dataset_size: inf
model: pix2pix
nThreads: 2
n_critic: 1
n_generator: 3
n_layers_D: 3
name: experiment_name
ndf: 64
ngf: 64
niter: 100
niter_decay: 100
no_augment: False
no_dropout: False
no_flip: False
no_html: False
no_img: False
no_lsgan: False
no_normalization: False
no_perceptual_loss: True
no_scaling: False
norm: batch_3d
norm_D: spectralbatch
norm_G: spectralbat

In [8]:
lr_logger = LearningRateLogger()

comet_logger = CometLogger(
    api_key="eyAsnp1KA7fXLxFMkEWKjhygS",
    project_name="kbp-challenge",
    workspace="mahruskazi"
)

CometLogger will be initialized in online mode


In [9]:
def objective(trial):
    # Filenames for each trial must be made unique in order to access each checkpoint.
    checkpoints_dir = '{}/checkpoints'.format(primary_directory)
    checkpoint_callback = ModelCheckpoint(
        filepath=checkpoints_dir,
        verbose=True,
        save_last=False,
        save_top_k=0,
        monitor='dose_score',
        mode='min'
    )

    # grad_batches = trial.suggest_int("grad_batches", 1, 16)
    grad_batches = 1
    weight_cliping_limit = trial.suggest_float("weight_cliping_limit", 0.001, 1.0)
    # opt.lr_D = trial.suggest_float("lr_D", 0.00001, 0.001)
    # opt.lr_G = trial.suggest_float("lr_G", 0.00001, 0.001)

    for key, value in trial.params.items():
      print("    {}: {}".format(key, value))

    # The default logger in PyTorch Lightning writes to event files to be consumed by
    # TensorBoard. We don't use any logger here as it requires us to implement several abstract
    # methods. Instead we setup a simple callback, that saves metrics from each validation step.
    metrics_callback = MetricsCallback()
    trainer = pl.Trainer(logger=comet_logger,
                         gpus=1,
                         checkpoint_callback=checkpoint_callback,
                         callbacks=[lr_logger, metrics_callback],
                         max_epochs=opt.epoch_count,
                         check_val_every_n_epoch=1,
                         num_sanity_val_steps=10,
                         limit_val_batches=1.0,
                         accumulate_grad_batches=grad_batches,
                         gradient_clip_val=weight_cliping_limit,
                         weights_summary='full',
                         early_stop_callback=PyTorchLightningPruningCallback(trial, monitor="dose_score"))

    model = Pix2PixModel(opt, model_name='hyper_search', stage='training')
    trainer.fit(model)

    return metrics_callback.metrics[-1]["dose_score"].item()

In [10]:
def end_of_trial(study, frozen_trial):
    # Save the study so far
    joblib.dump(study, 'study.pkl')

In [ ]:
pruner = optuna.pruners.MedianPruner()
resume_study = True

if resume_study:
    study = joblib.load('study.pkl')
    print('Best trial until now:')
    print(' Value: ', study.best_trial.value)
    print(' Params: ')
    for key, value in study.best_trial.params.items():
        print(f'    {key}: {value}')
else:
    study = optuna.create_study(direction="minimize", pruner=pruner)

study.optimize(objective, n_trials=100, gc_after_trial=True, callbacks=[end_of_trial])

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Best trial until now:
 Value:  8.08708891158013
 Params: 
    weight_cliping_limit: 0.850735371334598
    weight_cliping_limit: 0.18267318597926685


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


Generator using spectral normalization
Generator using spectral normalization
Generator using spectral normalization
Generator using spectral normalization
Generator using spectral normalization
Generator using spectral normalization
Generator using spectral normalization


/content/drive/My Drive/KBP_Challenge/src/models/networks.py:39: UserWarning:

nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.



Using spectral normalization
Using spectral normalization


...

Generator using spectral normalization
Generator using spectral normalization
Generator using spectral normalization
Generator using spectral normalization
Generator using spectral normalization
Generator using spectral normalization
Generator using spectral normalization
Using spectral normalization
Using spectral normalization



   | Name                                                                    | Type                    | Params
---------------------------------------------------------------------------------------------------------------------
0  | generator                                                               | UnetGenerator           | 167 M 
1  | generator.model                                                         | UnetSkipConnectionBlock | 167 M 
2  | generator.model.model                                                   | Sequential              | 167 M 
3  | generator.model.model.0                                                 | Conv3d                  | 4 K   
4  | generator.model.model.1                                                 | UnetSkipConnectionBlock | 167 M 
5  | generator.model.model.1.model                                           | Sequential              | 167 M 
6  | generator.model.model.1.model.0                                         | LeakyReLU         

Number of validation patients: 50


100%|██████████| 10/10 [00:06<00:00,  1.58it/s]


{'log': {'dose_score': tensor(39.6803, device='cuda:0', dtype=torch.float64), 'dvh_score': tensor(29.3316, device='cuda:0', dtype=torch.float64)}}
Number of training patients: 150
Number of validation patients: 50
